In [ ]:
import string
import random
import math
import re
from collections import Counter
import sys

# The training corpus is embedded directly in the script to avoid external files.
# This text is used to build a statistical model of the English language.
TRAINING_CORPUS = """
Alice's Adventures in Wonderland by Lewis Carroll

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought Alice 'without pictures or
conversations?'

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
running about, when suddenly a White Rabbit with pink eyes ran close by
her.
"""

class LanguageModel:
    """
    Encapsulates the statistical properties of English, learned from a corpus.
    This class is responsible for scoring text based on n-gram frequencies.
    """
    def __init__(self, corpus_text: str):
        """Initializes and trains the language model."""
        clean_text = self._normalize_text(corpus_text)
        self.trigram_log_probs = self._train_ngrams(clean_text, 3)
        self.bigram_log_probs = self._train_ngrams(clean_text, 2)
        # A penalty for n-grams not found in the corpus.
        self.log_floor = math.log(0.01 / len(clean_text))

    def _normalize_text(self, text: str) -> str:
        """Cleans text by uppercasing and removing non-alphabetic characters."""
        return re.sub(r'[^A-Z]', '', text.upper())

    def _train_ngrams(self, text: str, n: int) -> dict:
        """Calculates the log probability of each n-gram in the text."""
        counts = Counter(text[i:i+n] for i in range(len(text) - n + 1))
        total_ngrams = sum(counts.values())
        return {gram: math.log(count / total_ngrams) for gram, count in counts.items()}

    def score(self, text: str) -> float:
        """Scores a text based on how 'English-like' it is."""
        clean_text = self._normalize_text(text)
        score = 0.0
        # Combine bigram and trigram scores for a more accurate measure.
        for i in range(len(clean_text) - 1):
            score += self.bigram_log_probs.get(clean_text[i:i+2], self.log_floor)
        for i in range(len(clean_text) - 2):
            score += self.trigram_log_probs.get(clean_text[i:i+3], self.log_floor)
        return score

class CipherSolver:
    """
    The main cipher-cracking engine. Uses a LanguageModel to guide its
    search for the correct decryption key via a hill-climbing algorithm.
    """
    def __init__(self, language_model: LanguageModel):
        self.model = language_model
        self.alphabet = string.ascii_uppercase

    def apply_key(self, ciphertext: str, key: dict) -> str:
        """Applies a decryption key, preserving case and punctuation."""
        # Create a translation map for fast replacement
        key_map = str.maketrans(
            self.alphabet + self.alphabet.lower(),
            "".join([key.get(c, c) for c in self.alphabet]) +
            "".join([key.get(c, c).lower() for c in self.alphabet])
        )
        return ciphertext.translate(key_map)

    def solve(self, ciphertext: str, iterations: int = 3000, restarts: int = 70) -> (dict, str):
        """
        Cracks the cipher using a stochastic hill-climbing search.
        """
        best_key = {}
        best_score = -float('inf')

        print(f"\n🤖 Starting automated search ({restarts} restarts, {iterations} iterations each)...")

        for i in range(restarts):
            # Show progress to the user, indicating the script is working.
            print('.', end='', flush=True)

            # Start with an intelligent guess based on single-letter frequency.
            # This provides a much better starting point than a purely random key.
            parent_key = {}
            cipher_freq = Counter(c for c in ciphertext.upper() if c in self.alphabet)
            english_freq_order = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'
            sorted_cipher_chars = sorted(cipher_freq, key=cipher_freq.get, reverse=True)

            # Map the most frequent cipher chars to the most frequent English letters
            for idx, cipher_char in enumerate(sorted_cipher_chars):
                if idx < len(english_freq_order):
                    parent_key[cipher_char] = english_freq_order[idx]

            # Fill in any remaining alphabet characters randomly
            remaining_cipher_chars = [c for c in self.alphabet if c not in parent_key]
            remaining_plain_chars = [c for c in self.alphabet if c not in parent_key.values()]
            random.shuffle(remaining_plain_chars)
            for i, char in enumerate(remaining_cipher_chars):
                parent_key[char] = remaining_plain_chars[i]

            parent_score = self.model.score(self.apply_key(ciphertext, parent_key))

            # Iteratively improve the key by making small, random changes.
            for _ in range(iterations):
                child_key = parent_key.copy()
                char1, char2 = random.sample(self.alphabet, 2)
                # Swap two mappings
                child_key[char1], child_key[char2] = child_key[char2], child_key[char1]
                child_score = self.model.score(self.apply_key(ciphertext, child_key))

                # If the new key results in a more "English-like" text, keep it.
                if child_score > parent_score:
                    parent_key, parent_score = child_key, child_score

            # Keep track of the best key found across all restarts.
            if parent_score > best_score:
                best_score = parent_score
                best_key = parent_key

        print("\nSearch complete.")
        best_plaintext = self.apply_key(ciphertext, best_key)
        sorted_best_key = dict(sorted(best_key.items()))
        return sorted_best_key, best_plaintext

# --- Main Execution Block ---
if __name__ == "__main__":

    # 1. Initialize the tools using the embedded corpus
    print("🧠 Training language model (one-time setup)...")
    model = LanguageModel(TRAINING_CORPUS)
    solver = CipherSolver(model)

    # 2. Prompt the user for input using a more compatible method
    print("\n--- Aristocrat Cipher Solver ---")
    print("Please paste the ciphertext you want to solve.")
    print("After pasting, press ENTER on a new, empty line to start.")

    lines = []
    while True:
        try:
            line = input()
            # An empty line from the user signifies the end of the input
            if line:
                lines.append(line)
            else:
                break
        except EOFError:
            # Handles Ctrl+D/Ctrl+Z as a fallback for terminal users
            break

    ciphertext = "\n".join(lines)

    # 3. Check for valid input and run the solver
    if not ciphertext.strip():
        print("\nNo input received. Exiting.")
    else:
        final_key, final_plaintext = solver.solve(ciphertext)

        # 4. Display the final solution, prioritizing the decoded text.
        print("\n" + "="*50)
        print("🎉                 SOLUTION FOUND                 🎉")
        print("="*50)

        print("\n[+] Decrypted Message:")
        print(final_plaintext)

        print("\n[+] Final Decryption Key:")
        key_items = [f"{c}→{p}" for c, p in final_key.items()]
        # Print the key in two neat rows for readability
        print("  " + "  ".join(key_items[:13]))
        print("  " + "  ".join(key_items[13:]))